In [19]:
# PyTorch张量统计运算学习笔记
# 涵盖:求和、均值、方差、最大最小值、排序、去重等操作
import torch
from sympy.abc import Q

In [20]:
# 创建测试张量
# shape:(3,2,4) 可理解为3个2x4的矩阵
# .float()转换为float32类型,因为统计运算通常需要浮点数
# 应用:用于测试多维张量上的统计操作
tensor1 = torch.randint(1, 10, (3, 2, 4)).float()
print(tensor1)

tensor([[[5., 5., 3., 2.],
         [7., 9., 6., 3.]],

        [[7., 2., 3., 8.],
         [2., 2., 4., 7.]],

        [[3., 6., 7., 5.],
         [6., 5., 4., 4.]]])


In [21]:
# sum求和运算
# .sum(): 对所有元素求和,返回标量
# .sum(dim=k): 沿着第k维求和,该维度被消除
# .sum(dim=(i,j)): 沿着多个维度求和
# 应用:计算总损失、batch内求和、注意力权重归一化

# 示例:tensor1.shape=(3,2,4), 共24个元素
print(tensor1.sum())  # 全局求和:所有24个数相加

# dim=0求和:沿着第0维(3个矩阵)求和
# 结果:3个(2,4)矩阵对应位置相加→(2,4)
# 理解:(3,2,4)→消除dim0→(2,4)
print(tensor1.sum(dim=0))  # shape:(2,4)

# dim=(0,2)求和:同时沿着0和2维求和
# 结果:(3,2,4)→消除dim0和dim2→(2,)
# 计算:对于结果的每一行,将所有3个矩阵的该行的4个元素都加起来
print(tensor1.sum(dim=(0, 2)))  # shape:(2,)

tensor(115.)
tensor([[15., 13., 13., 15.],
        [15., 16., 14., 14.]])
tensor([56., 59.])


In [22]:
# mean求均值
# 用法与sum相同,但返回的是平均值
# 公式:mean = sum / count
# 应用:batch平均损失、均值池化(Global Average Pooling)

print(tensor1.mean())  # 全局均值:sum(24个元素)/24

# dim=0均值:沿着0维求平均
# 计算:3个(2,4)矩阵对应位置的3个值求平均
print(tensor1.mean(dim=0))  # shape:(2,4)

# dim=(0,2)均值:同时沿着0和2维求平均
# 计算:结果每个元素是3*4=12个数的平均值
print(tensor1.mean(dim=(0, 2)))  # shape:(2,)

tensor(4.7917)
tensor([[5.0000, 4.3333, 4.3333, 5.0000],
        [5.0000, 5.3333, 4.6667, 4.6667]])
tensor([4.6667, 4.9167])


In [23]:
# std求标准差
# 标准差衡量数据的离散程度
# 公式:std = sqrt(mean((x - mean(x))^2))
# 应用:Batch Normalization、梯度裁剪阈值设置、特征标准化

print(tensor1.std())  # 全局标准差

# dim=0标准差:每个位置上3个值的标准差
# 示例:tensor1[:,0,0]的3个值的标准差→结果[0,0]
print(tensor1.std(dim=0))  # shape:(2,4)

# dim=(0,2)标准差:
print(tensor1.std(dim=(0, 2)))  # shape:(2,)

tensor(2.0637)
tensor([[2.0000, 2.0817, 2.3094, 3.0000],
        [2.6458, 3.5119, 1.1547, 2.0817]])
tensor([2.0597, 2.1515])


In [24]:
# max、min求最大最小值对应位置的3个值求平均
print(tensor1.mean(dim=0))  # shape:(2,4)

# dim=(0,2)均值:同时沿着0和2维求平均
# 计算:结果每个元素是3*4=12个数的平均值
print(tensor1.mean(dim=(0, 2)))  # shape:(2,)
# std求标准差
# 标准差衡量数据的离散程度
# 公式:std = sqrt(mean((x - mean(x))^2))
# 应用:Batch Normalization、梯度裁剪阈值设置、特征标准化

# .max(): 返回单个标量(全局最大值)
# .max(dim=k): 返回(values, indices)两个张量
#   - values: 每个位置的最大值
#   - indices: 最大值在dim=k维度上的索引位置
# 应用:分类任务取预测类别、池化层、TopK准确率

print(tensor1.max())  # 全局最大值(单个数)
print(tensor1.min())  # 全局最小值(单个数)

# dim=0最大值:
# values[i,j]: 3个矩阵中位置(i,j)的最大值
# indices[i,j]: 该最大值来自第几个矩阵(0/1/2)
# 示例:若values[0,1]=7且indices[0,1]=1, 表示tensor1[1,0,1]=7是最大的
print(tensor1.max(dim=0))  # 返回values(2,4)和indices(2,4)

# dim=0最小值:逻辑同max
print(tensor1.min(dim=0))  # 返回values(2,4)和indices(2,4)

tensor([[5.0000, 4.3333, 4.3333, 5.0000],
        [5.0000, 5.3333, 4.6667, 4.6667]])
tensor([4.6667, 4.9167])
tensor(9.)
tensor(2.)
torch.return_types.max(
values=tensor([[7., 6., 7., 8.],
        [7., 9., 6., 7.]]),
indices=tensor([[1, 2, 2, 1],
        [0, 0, 0, 1]]))
torch.return_types.min(
values=tensor([[3., 2., 3., 2.],
        [2., 2., 4., 3.]]),
indices=tensor([[2, 1, 0, 0],
        [1, 1, 1, 0]]))


In [25]:
# argmin求最小值的索引位置
# 将张量展平为一维后,返回最小元素的位置索引
# 示例:tensor1共(3,2,4)=24个元素,展平后索引为0-23
# 返回Tensor(22)表示最小值在第22个位置(从0开始)
# 对应关系:索引22 → (22//8, 22%8//4, 22%4) = (2,2,2) → tensor1[2,2,2]
# 应用:查找极值位置、负样本挖掘
print(tensor1.argmin())  # 返回展平后的一维索引

tensor(3)


In [26]:
# unique去重
# 返回排序后的唯一值列表
# 示例:[1,3,2,3,1] → [1,2,3]
# 应用:统计类别数、检查标签范围、数据探索性分析
print(torch.unique(tensor1))  # 返回[1,2,3,4,5,6,7,8,9](排序)

tensor([2., 3., 4., 5., 6., 7., 8., 9.])


In [27]:
# sort排序
# 默认沿最后一个维度升序排序
# 返回(values, indices):
#   - values: 排序后的张量
#   - indices: 原始元素在排序前的索引位置
# 应用:TopK选择、排序池化、中位数计算

# 示例:tensor1.shape=(3,2,4),沿最后一维(4个元素)排序
# 结果:每个(2,4)矩阵的每一行的四个元素升序排列
# indices记录排序后元素在原行中的位置
# 如果原行=[5,2,8,3],排序后=[2,3,5,8],indices=[1,3,0,2]
print(tensor1.sort())  # 默认dim=-1(最后一维)

torch.return_types.sort(
values=tensor([[[2., 3., 5., 5.],
         [3., 6., 7., 9.]],

        [[2., 3., 7., 8.],
         [2., 2., 4., 7.]],

        [[3., 5., 6., 7.],
         [4., 4., 5., 6.]]]),
indices=tensor([[[3, 2, 0, 1],
         [3, 2, 0, 1]],

        [[1, 2, 0, 3],
         [0, 1, 2, 3]],

        [[0, 3, 1, 2],
         [2, 3, 1, 0]]]))
